In [1]:
import pandas as pd
import numpy as np
import glob
import os
import datetime

In [2]:
sales_file_list=glob.glob("/home/jian/Projects/Saatva/Transaction_Data/New Klipfolio with phone num and rep name/"+"*.csv")
sales_file_list=sorted(sales_file_list)

In [3]:
sales_file_df=pd.DataFrame({"file":sales_file_list},index=[x for x in range(len(sales_file_list))])
sales_file_df['Date']=sales_file_df['file'].apply(lambda x: datetime.datetime.strptime(x[len(x)-12:len(x)-4],"%Y%m%d"))
sales_file_df=sales_file_df[sales_file_df['Date']>=datetime.date(2018,10,1)].reset_index()
sales_file_df=sales_file_df.sort_values('Date',ascending=False)
del sales_file_df['index']
sales_file_df=sales_file_df.reset_index()
del sales_file_df['index']
sales_file_df.head(2)
header=pd.read_csv(sales_file_df['file'][0],dtype=str).columns.tolist()

In [4]:
sales_combined=pd.DataFrame(dict(zip(header,[np.nan]*len(header))),index=[])

for i in range(len(sales_file_df)):
    file=sales_file_df['file'][i]
    df=pd.read_csv(file,dtype=str)
    df=df[df['Brand']=="Saatva"]
    df['Date']=df['Date'].apply(lambda x: datetime.datetime.strptime(x,"%d-%b-%Y").date())
    df_date_max=df['Date'].max()
    df_date_min=df['Date'].min()
    df=df[~df['Date'].isin(sales_combined['Date'].tolist())]
    df=df[(df['Date']>df_date_min) & (df['Date']<df_date_max)]
    sales_combined=sales_combined.append(df)

sales_combined=sales_combined[(sales_combined['Date']>=datetime.date(2018,9,1)) & (sales_combined['Date']<=datetime.date(2018,11,30))]
original_data=sales_combined.copy()

In [5]:
sales_combined['Shipping Zip']=sales_combined['Shipping Zip'].apply(lambda x: x.split("-")[0].zfill(5))
sales_combined=sales_combined.rename(columns={"Shipping Zip":"zip_cd"})

In [6]:
Zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,usecols=[0,2],skiprows=1)
Zip_DMA=Zip_DMA.rename(columns={"CODE":"zip_cd","NAME":"DMA"})
Zip_DMA=Zip_DMA.drop_duplicates()

In [7]:
sales_combined=pd.merge(sales_combined,Zip_DMA,on="zip_cd",how="left")
sales_combined['DMA']=sales_combined['DMA'].fillna("Not_In_DMA")
sales_combined['Product Revenue']=sales_combined['Product Revenue'].astype(float)
sales_combined_by_DMA_sales=sales_combined.groupby(['DMA','Order Type'])['Product Revenue'].sum().to_frame().reset_index()
sales_combined_by_DMA_sales=sales_combined_by_DMA_sales.rename(columns={"Product Revenue":"sales"})
sales_combined_by_DMA_trans=sales_combined.groupby(['DMA','Order Type'])['Product Revenue'].count().to_frame().reset_index()
sales_combined_by_DMA_trans=sales_combined_by_DMA_trans.rename(columns={"Product Revenue":"trans"})


In [8]:
sales_combined_by_DMA_sales_online=sales_combined_by_DMA_sales[sales_combined_by_DMA_sales['Order Type']=="online"]
sales_combined_by_DMA_sales_online=sales_combined_by_DMA_sales_online[['DMA','sales']].rename(columns={"sales":"online_sales"})
sales_combined_by_DMA_sales_phone=sales_combined_by_DMA_sales[sales_combined_by_DMA_sales['Order Type']=="phone"]
sales_combined_by_DMA_sales_phone=sales_combined_by_DMA_sales_phone[['DMA','sales']].rename(columns={"sales":"phone_sales"})

sales_combined_by_DMA_trans_online=sales_combined_by_DMA_trans[sales_combined_by_DMA_trans['Order Type']=="online"]
sales_combined_by_DMA_trans_online=sales_combined_by_DMA_trans_online[['DMA','trans']].rename(columns={"trans":"online_trans"})
sales_combined_by_DMA_trans_phone=sales_combined_by_DMA_trans[sales_combined_by_DMA_trans['Order Type']=="phone"]
sales_combined_by_DMA_trans_phone=sales_combined_by_DMA_trans_phone[['DMA','trans']].rename(columns={"trans":"phone_trans"})



In [9]:
output=pd.merge(sales_combined_by_DMA_sales_online,sales_combined_by_DMA_sales_phone,on="DMA",how="outer")
output=pd.merge(output,sales_combined_by_DMA_trans_online,on="DMA",how="outer")
output=pd.merge(output,sales_combined_by_DMA_trans_phone,on="DMA",how="outer")
output=output.fillna(0)
output['total_sales']=output['online_sales']+output['phone_sales']
output['total_trans']=output['online_trans']+output['phone_trans']
output=output.sort_values(['total_sales','total_trans'],ascending=[False,False])

In [10]:
writer=pd.ExcelWriter("/home/jian/Projects/Saatva/Analysis/Sep_to_Nov_Top_15_DMAs/Saatva_Top_15_DMAs_Sep_to_Nov_2018_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output.to_excel(writer,"Sales_by_DMA",index=False)
original_data.to_excel(writer,"Sales_Data",index=False)
Zip_DMA.to_excel(writer,"Zip_DMA_Mapping",index=False)
writer.save()